## Demo of the Adaptive Ground Point Filtering Library

This notebook gives a tour of the capabilities of `filteradapt`. It is continuously updated while we develop the library. You can execute the code in each cell by pressing `Ctrl+Enter`. To start your own project based on `filteradapt`, you can either edit this notebook or start a new one.

The first thing to do in a Jupyter notebook that uses `filteradapt` is to import the library:

In [ ]:
import adaptivefiltering

The next step you will typically do is to construct a dataset object by reading a LAS/LAZ file:

In [ ]:
#ds = adaptivefiltering.DataSet("/home/gwydion/SSC/ThingstaetteDataset/uls_thingstaette.las")
ds = adaptivefiltering.DataSet("data/500k_NZ20_Westport.laz")

In [ ]:
#ds.show()

In [ ]:
ds.show_mesh(resolution=2)


In [ ]:
import pdal
import json
import numpy as np
import ipyleaflet
import ipywidgets


The data set can be visualized in the notebook through its `show()` method:

In [ ]:
hexbin_json = [
    ds.filename,
    
{
        "type": "filters.reprojection",
        "out_srs":"EPSG:4326"    
        },
    {
        "type": "filters.hexbin"
    },
    
]


In [ ]:
hexbin_pipeline = pdal.Pipeline(json.dumps(hexbin_json))
hexbin_count = hexbin_pipeline.execute()
hexbin_geojson = json.loads(hexbin_pipeline.metadata)["metadata"]["filters.hexbin"]["boundary_json"]
#get the previous coordinate representation
dataset_transformation = json.loads(hexbin_pipeline.metadata)["metadata"]["readers.las"]["comp_spatialreference"].split(r'GEOGCS["')[1].split(r'",')[0]
print(dataset_transformation)

In [ ]:
boundary_coordinates = (hexbin_geojson["coordinates"][0])
print(boundary_coordinates)
coordinates_mean = np.mean(boundary_coordinates,axis=0)

In [ ]:
polygon_boundary = ipyleaflet.Polygon(
    locations=boundary_coordinates,
    color="gray",
    opacity=0.9
)

m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldImagery, 
                    center=(coordinates_mean[0], 
                    coordinates_mean[1]), 
                    crs=ipyleaflet.projections.EPSG3857,
                    scroll_wheel_zoom=True,
                    max_zoom=20)
m.add_layer(polygon_boundary)

zoom_slider = ipywidgets.IntSlider(description='Zoom level:', min=0, max=20, value=16)
ipywidgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position='topright')
#m.add_control(widget_control1)



color_picker = ipywidgets.ColorPicker(description='Pick a color:')

draw_control = ipyleaflet.DrawControl(widget = color_picker)

draw_control.polygon = {
    "shapeOptions": {
        "fillColor": color_picker.get_interact_value()
,
        "color": color_picker.get_interact_value()
,
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
def update_draw_control_color( *args, **kwargs):
        m.remove_control(draw_control)
        draw_control.polygon = {"shapeOptions": {
                        "fillColor": color_picker.get_interact_value(),
                        "color": color_picker.get_interact_value(),
                        "fillOpacity": 0.4
                        }}
        m.add_control(draw_control)

widget_control2 = ipyleaflet.WidgetControl(widget=color_picker, position='bottomright')
draw_control.on_draw(update_draw_control_color)

m.add_control(widget_control2)
m.add_control(draw_control)
m

In [ ]:
class interactive_map:

    def __init__(self, boundary_coordinates):
        polygon_boundary = ipyleaflet.Polygon(
            locations=boundary_coordinates,
            color="gray",
            opacity=0.9
        )

        self.m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldImagery, 
                            center=(coordinates_mean[0], 
                            coordinates_mean[1]), 
                            crs=ipyleaflet.projections.EPSG3857,
                            scroll_wheel_zoom=True,
                            max_zoom=20)
        self.m.add_layer(polygon_boundary)

        zoom_slider = ipywidgets.IntSlider(description='Zoom level:', min=0, max=20, value=16)
        ipywidgets.jslink((zoom_slider, 'value'), (self.m, 'zoom'))
        

        color_picker = ipywidgets.ColorPicker(description='Pick a color:')

        draw_control = ipyleaflet.DrawControl()

        draw_control.polygon = {
            "shapeOptions": {
                "fillColor": color_picker.get_interact_value()
        ,
                "color": color_picker.get_interact_value()
        ,
                "fillOpacity": 1.0
            },
            "drawError": {
                "color": "#dd253b",
                "message": "Oups!"
            },
            "allowIntersection": False
        }
        color_control = ipyleaflet.WidgetControl(widget=color_picker, position='bottomright')
        self.m.add_control(color_control)
        self.m.add_control(draw_control)
        ipywidgets.jslink((color_picker, 'value'), (draw_control.polygon["shapeOptions"], 'color'))


    def update_draw_control_color(self, *args, **kwargs):
        self.m.remove_control(draw_control)
        draw_control.polygon = {"shapeOptions": {
                        "fillColor": color_picker.get_interact_value(),
                        "color": color_picker.get_interact_value(),
                        "fillOpacity": 0.4
                        }}
        self.m.add_control(draw_control)
        
    def show(self):
        return self.m

In [ ]:
test_map = interactive_map(boundary_coordinates)
test_map.show()

In [ ]:

polygon = ipyleaflet.Polygon(
    locations=[(42, -49), (43, -49), (43, -48)],
    color="green",
    fill_color="green",
    transform=True
)

m = ipyleaflet.Map(center=(42.5531, -48.6914), zoom=6)
m.add_layer(polygon);

m